In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [50]:
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br")

In [51]:
# Initialize a list to store table data
all_data = []
# Wait for the table to be present
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#segment > option:nth-child(2)"))
)

filter_button = driver.find_element(By.CSS_SELECTOR, "#segment > option:nth-child(2)")
filter_button.click()


while True:
    # Wait for the table to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table"))
    )

    # Find the table
    table = driver.find_element(By.CSS_SELECTOR, "table")

    # Get table headers
    headers = [header.text for header in table.find_elements(By.CSS_SELECTOR, "thead th") if header.text != '%Setor']

    # Get rows data
    rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")
    for row in rows:
        cells = row.find_elements(By.CSS_SELECTOR, "td")
        row_data = [cell.text for cell in cells]
        all_data.append(row_data)

    # Check if there is a next page button
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "#listing_pagination > pagination-template > ul > li.pagination-next > a")
        if "disabled" in next_button.get_attribute("class"):
            break  # Exit loop if no more pages
        next_button.click()  # Click next page
    except:
        break  # Exit loop if no next button found

# Convert data to DataFrame for better handling
df = pd.DataFrame(all_data, columns=headers)
df.head()

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON NM,1.482.105.837,"3,291","3,291"
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,734.632.601,"1,443","1,443"
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,332.825.777,"0,124","2,027"
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,995.335.937,"0,593","2,027"
4,Bens Indls/Transporte,RAIL3,RUMO S.A.,ON NM,1.218.352.541,"1,310","2,027"


In [60]:
import boto3
from io import StringIO
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

# Initialize a session using Amazon S3
s3 = boto3.client('s3')

# Convert DataFrame to CSV format in memory
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)

# Define the name of your bucket and the file to upload
bucket_name = 'marcelofelixfiap'
object_name = 'b3_pregao_D-1.csv'  # S3 object name (where the file will be stored)
file_like_object = StringIO(df.to_csv(object_name))

try:
    # Upload the file
#     s3.upload_file(file_name, bucket_name, object_name)
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=csv_buffer.getvalue(), ContentType='text/csv')
    print(f"File '{file_name}' uploaded to bucket '{bucket_name}' as '{object_name}'.")
except FileNotFoundError:
    print(f"The file '{file_name}' was not found.")
except NoCredentialsError:
    print("Credentials not available.")
except PartialCredentialsError:
    print("Incomplete credentials provided.")
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.
